<a href="https://colab.research.google.com/github/EnricoMiche1210760/toxic_comments_filter/blob/main/progetto_toxic_comments_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project: Toxic Comment Filter

Costruire un modello in grado di filtrare i commenti degli utenti in base al grado di dannosità del linguaggio:
1. Preprocessare il testo eliminando l'insieme di token che non danno contributo significativo a livello semantico
2. Trasformare il corpus testuale in sequenze
3. Costruire un modello di Deep Learning comprendente dei layer ricorrenti per un task di classificazione multilabel
4. In prediction time, il modello deve ritornare un vettore contenente un 1 o uno 0 in corrispondenza di ogni label presente nel dataset (toxic,	severe_toxic,	obscene,	threat,	insult,	identity_hate). In questo modo, un commento non dannoso sarà classificato da un vettore di soli 0 [0,0,0,0,0,0]. Al contrario, un commento pericoloso presenterà almeno un 1 tra le 6 labels.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config Completer.use_jedi = False


In [ ]:
import pandas as pd
BASE_URL = "https://s3.eu-west-3.amazonaws.com/profession.ai/datasets/"
df = pd.read_csv(BASE_URL+"Filter_Toxic_Comments_dataset.csv")

In [ ]:
df.head()

In [ ]:
df[df['sum_injurious']==2]

In [ ]:
len(df)

In [ ]:
import nltk
import numpy as np

df_text = np.array(df['comment_text'])
type(df_text)

preprocessed_text = []
preprocessed_text.append(' '.join([word for i in range(len(df_text)) \
                                    for word in nltk.word_tokenize(df_text[i].lower()) \
                                    if word.isalpha()]))

In [ ]:
df_text[0]
preprocessed_text[0]